# Final Project Template

<div class="alert alert-block alert-danger">
Delete all cells including this one before submitting your final notebook.
</div>

<img src="https://s3.amazonaws.com/ByC_logo_prod/unit-26963.jpg" alt="Alt text that describes the graphic" width="600" />

# Title

Add the title of your project and delete the default one.

# Table of contents

- [Abstract](#Abstract)
- [Introduction](#Introduction)
- [1. The problem](#1.-The-problem)
- [2. Purpose of the study](#2.-Purpose-of-the-study)
- [3. Project description](#3.-Project-description)
    - [3.1 Hypothesis](#3.1-Hypothesis)
    - [3.2 Workflow](#3.2-Workflow)
    - [3.3 Methods](#3.3-Methods)
    - [3.4Analysis](#3.4-Analysis)
    - [3.5 Results](#3.5-Results)
- [4. Implementation (optional)](#4.-Implementation-(optional))
- [Conclusion](#Conclusion)
- [References](#References)

In [1]:
import warnings
import pandas as pd
import numpy as np
import time
import datetime
import os
import seaborn as sns
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

pd.set_option('display.max_columns', None)
%matplotlib inline

warnings.filterwarnings('ignore')


In [2]:
folder = 'mpls_data/mpls_police_data/'
df = pd.read_csv(folder + 'police_incidents_2021_pims.csv', parse_dates=['beginDate'])

def convertIntToTime(t):
    s = t.astype(str).str.zfill(4) + '00'
    return pd.to_datetime(s.str[:2] + ':' + s.str[2:4] + ':' + s.str[4:]).dt.time

df['beginTimeNew']  = convertIntToTime(df['beginTime'])
df['beginDateTime'] = pd.to_datetime(df['beginDate'].dt.date.astype(str) + df['beginTimeNew'].astype(str), format = '%Y-%m-%d%H:%M:%S')

print(df['beginDateTime'])
df.head()


0       2021-07-29 02:10:00
1       2021-06-01 00:00:00
2       2021-07-16 05:34:00
3       2021-07-17 02:10:00
4       2021-07-17 20:45:00
                ...        
14343   2021-06-21 21:00:00
14344   2021-06-21 22:00:00
14345   2021-06-22 13:30:00
14346   2021-06-18 20:20:00
14347   2021-06-20 23:30:00
Name: beginDateTime, Length: 14348, dtype: datetime64[ns]


,X,Y,publicaddress,caseNumber,precinct,reportedDate,reportedTime,beginDate,reportedDateTime,beginTime,offense,description,UCRCode,enteredDate,centergbsid,centerLong,centerLat,centerX,centerY,neighborhood,lastchanged,LastUpdateDateETL,OBJECTID,beginTimeNew,beginDateTime
0,-93.261215,44.992450,0004XX 2ND ST NE,MP2021168582,2,2021/07/29 00:00:00+00,210,2021-07-29 00:00:00+00:00,2021/07/29 02:10:00+00,210,ARSON,ARSON,10.0,2021/07/29 00:00:00+00,16554.0,-93.261218,44.992454,-10381791,5620333,St. Anthony West,2021/08/06 00:00:00+00,2021/08/07 07:10:05+00,1,02:10:00,2021-07-29 02:10:00
1,-93.254370,44.951943,0014XX 28TH ST E,MP2021168739,3,2021/07/28 00:00:00+00,2345,2021-06-01 00:00:00+00:00,2021/07/28 23:45:00+00,0,CSCR,CSC - RAPE,3.0,2021/07/29 00:00:00+00,16882.0,-93.254378,44.951949,-10381029,5613959,Midtown Phillips,2021/08/06 00:00:00+00,2021/08/07 07:10:05+00,2,00:00:00,2021-06-01 00:00:00
2,-93.267072,44.978637,NaN,MP2021157179,1,2021/07/20 00:00:00+00,2315,2021-07-16 00:00:00+00:00,2021/07/20 23:15:00+00,534,THEFT,OTHER THEFT,7.0,2021/07/21 00:00:00+00,NaN,-93.267070,44.978640,-10382443,5618159,Downtown West,2021/07/21 00:00:00+00,2021/07/22 07:05:01+00,3,05:34:00,2021-07-16 05:34:00
3,-93.272220,44.948351,0003XX LAKE ST E,MP2021157976,3,2021/07/17 00:00:00+00,306,2021-07-17 00:00:00+00:00,2021/07/17 03:06:00+00,210,ASLT3,ASLT-SGNFCNT BDLY HM,5.0,2021/07/17 00:00:00+00,19102.0,-93.272225,44.948354,-10383016,5613394,Central,2021/07/30 00:00:00+00,2021/08/02 10:06:09+00,4,02:10:00,2021-07-17 02:10:00
4,-93.235694,44.981751,0013XX 5TH ST SE,MP2021158593,2,2021/07/17 00:00:00+00,2155,2021-07-17 00:00:00+00:00,2021/07/17 21:55:00+00,2045,ROBPER,ROBBERY OF PERSON,4.0,2021/07/19 00:00:00+00,16637.0,-93.235695,44.981751,-10378950,5618649,Marcy Holmes,2021/07/21 00:00:00+00,2021/07/22 07:05:01+00,5,20:45:00,2021-07-17 20:45:00


In [3]:
def import_pims_data(path):

    df = pd.read_csv(path, parse_dates=['beginDate'])
    df['beginTimeNew']  = convertIntToTime(df['beginTime'])
    df['Dates'] = pd.to_datetime(df['beginDate'].dt.date.astype(str) + df['beginTimeNew'].astype(str), format = '%Y-%m-%d%H:%M:%S')
    df.drop(columns={'X', 'Y', 'caseNumber','reportedDate', 'reportedTime', 'beginDate', 'reportedDateTime', 'beginTime', 'enteredDate', 'centergbsid', 'centerX', 'centerY', 'lastchanged', 'LastUpdateDateETL',  'OBJECTID', 'beginTimeNew'}, inplace=True)
    df.rename(columns={'publicaddress':'Address','centerLat': 'Lat', 'centerLong': 'Long', 'description': 'Description', 'offense': 'Offense', 'precinct': 'Precinct', 'reportedDateTime': 'ReportedDateTime', 'neighborhood': 'Neighborhood'}, inplace=True)
    df = df[['Dates','Address', 'Lat', 'Long', 'Neighborhood', 'Precinct', 'Offense', 'Description', 'UCRCode']] 
    #df['UCRCode'] =df['UCRCode'].astype(int)
    df['Dates'] = pd.to_datetime(df['Dates'], utc=False)
    return df

def convertIntToTime(t):
    s = t.astype(str).str.zfill(4) + '00'
    return pd.to_datetime(s.str[:2] + ':' + s.str[2:4] + ':' + s.str[4:]).dt.time


folder = 'mpls_data/mpls_police_data/'

c18 = import_pims_data(folder + 'police_incidents_2018_pims.csv')
c19 = import_pims_data(folder + 'police_incidents_2019_pims.csv')
c20 = import_pims_data(folder + 'police_incidents_2020_pims.csv')
c21 = import_pims_data(folder + 'police_incidents_2021_pims.csv')

frames = [c18, c19, c20, c21]
df_pims = pd.concat(frames)

df_pims.head()
df_pims.info

<bound method DataFrame.info of                     Dates                 Address        Lat       Long  \
0     2018-06-05 20:00:00         0012XX YALE PL   44.972645 -93.279485   
1     2018-06-07 12:10:00        0001XX LAKE ST W  44.948347 -93.280523   
2     2018-06-07 17:37:00  0003XX NICOLLET MALL    44.980281 -93.269393   
3     2018-06-07 21:00:00        0041XX 3RD AVE S  44.927773 -93.272738   
4     2018-06-07 13:30:00         00003X 9TH ST N  44.976774 -93.277360   
...                   ...                     ...        ...        ...   
14343 2021-06-21 21:00:00   0045XX MINNEHAHA AVE   44.920621 -93.215306   
14344 2021-06-21 22:00:00         0003XX 1ST ST N  44.986581 -93.270715   
14345 2021-06-22 13:30:00   00002X FRANKLIN AVE E  44.962689 -93.277185   
14346 2021-06-18 20:20:00     0026XX DUPONT AVE S  44.954645 -93.293151   
14347 2021-06-20 23:30:00       0052XX 40TH AVE S  44.908098 -93.215215   

          Neighborhood Precinct   Offense                     Descr

In [4]:
def import_caprs_data(path):
    #df = []
    df = pd.read_csv(path)
    #df.drop(columns={'caseNumber','reportedDate', 'reportedTime', 'beginDate', 'beginTime', 'enteredDate', 'centergbsid', 'centerX', 'centerY', 'lastchanged', 'LastUpdateDateETL',  'OBJECTID', 'beginTimeNew'}, inplace=True)
    df.drop(columns={'FID', 'ControlNbr', 'CCN', 'ReportedDate', 'Time', 'EnteredDate', 'GBSID', 'X', 'Y', 'LastChanged', 'LastUpdateDate' }, inplace=True)
    df.rename(columns={'BeginDate':'Dates', 'PublicAddress': 'Address'}, inplace=True)
    df = df[['Dates','Address', 'Lat', 'Long', 'Neighborhood', 'Precinct', 'Offense', 'Description', 'UCRCode']]
    #print(df.columns)
    df['Dates'] = pd.to_datetime(df['Dates'], utc=False)
    return df


folder = 'mpls_data/mpls_police_data/'
path = folder + 'police_incidents_2010_caprs.csv'

c10 = import_caprs_data(folder + 'police_incidents_2010_caprs.csv')
c11 = import_caprs_data(folder + 'police_incidents_2011_caprs.csv')
c12 = import_caprs_data(folder + 'police_incidents_2012_caprs.csv')
c13 = import_caprs_data(folder + 'police_incidents_2013_caprs.csv')
c14 = import_caprs_data(folder + 'police_incidents_2014_caprs.csv')
c15 = import_caprs_data(folder + 'police_incidents_2015_caprs.csv')
c16 = import_caprs_data(folder + 'police_incidents_2016_caprs.csv')
c17 = import_caprs_data(folder + 'police_incidents_2017_caprs.csv')
c18 = import_caprs_data(folder + 'police_incidents_2018_caprs.csv')

frames = [c10, c11, c12, c13, c14, c15, c16, c17, c18]
df_caprs = pd.concat(frames)

df_caprs.head()
df_caprs.shape

(176877, 9)

In [5]:
# concatenate df's together and 
frames = [df_caprs, df_pims]
df = pd.concat(frames)
df.head()

,Dates,Address,Lat,Long,Neighborhood,Precinct,Offense,Description,UCRCode
0,2010-01-01 00:30:00,0056XX Lyndale AV S,44.900291,-93.288239,WINDOM,5.0,CSCR,Crim Sex Cond-rape,3.0
1,2010-01-01 01:05:00,0001XX 62 ST W,44.890636,-93.280041,WINDOM,5.0,ROBPAG,Robbery Per Agg,4.0
2,2010-01-01 04:18:00,0039XX 5 AV S,44.931448,-93.268841,BRYANT,3.0,BURGD,Burglary Of Dwelling,6.0
3,2010-01-01 03:00:00,0013XX Nicollet AV S,44.970506,-93.277714,LORING PARK,1.0,ASLT2,Asslt W/dngrs Weapon,5.0
4,2010-01-01 02:23:00,0034XX Oliver AV N,45.017746,-93.306988,FOLWELL,4.0,BURGD,Burglary Of Dwelling,6.0


## Data Cleansing

In [6]:
# check for duplicate records
df.duplicated(subset=None, keep='first').sum()
#df.shape

1161

In [7]:

# # strip date and time and hour from timestamp
# df['Time'] = df['Dates'].apply( lambda d : d.time() )
# df['Date'] = df['Dates'].apply( lambda d : d.date() )
# df['Year'] = df['Dates'].apply( lambda d : d.year)
# df['Month'] = df['Dates'].apply( lambda d : d.month)
# df['Month_Name'] = df['Dates'].apply( lambda d : d.strftime('%B'))
# df['DayOfWeek_Num'] = df['Dates'].apply( lambda d : d.dayofweek)
# df['DayOfWeek'] = df['Dates'].dt.day_name()
# df['Hour'] = df['Dates'].apply( lambda d : d.hour)

# #df['Year_Month'] = df['Year'].astype(str) + '_' + df['Month'].astype(str)

# # make date a date char type
# df['Date'] = pd.to_datetime(df['Date'])

# # remove whitespace from Offense and Precinct
# df['Offense'] = df['Offense'].str.strip()

# # change AUTOTH to GTA to make more clear
# df['Offense']=df['Offense'].apply(lambda x:(x.replace('AUTOTH','GTA')))
# # combine other autothefts into one as they are all similar
# df['Offense']=df['Offense'].apply(lambda x:(x.replace('MVTHFT', 'TFMV').replace('TMVP', 'TFMV')))

# # remove punctuation from neighborhood column
# df['Neighborhood'] = df['Neighborhood'].str.replace(r"[\"\',]", '')

# # remove na's (exist in precinct and neighborhood)
# df = df.dropna()

# # remove UI, 18 and 99 from precinct
# df = df[df.Precinct != 'UI    ']
# df = df[df.Precinct != 18]
# df = df[df.Precinct != 99]

# # convert precinct column to int
# df['Precinct'] = df['Precinct'].astype(int)

# # convert neighborhood to upper
# df['Neighborhood'] = df['Neighborhood'].str.upper()

# df_incidents = df[['Dates', 'Address', 'Long', 'Lat', 'Neighborhood', 'Precinct', 'Offense', 'Description', 'UCRCode', 'DayOfWeek']]

# df_incidents.tail()

In [8]:
df_incidents.shape

path = os.getcwd()
print(path)
df_incidents.to_csv(os.getcwd() +'/minneapolis_incidents.csv')

NameError: name 'df_incidents' is not defined

In [ ]:
# load in weather data
from datetime import datetime
df = pd.read_csv('mpls_Data/mpls_weather_data/mpls_weather_data.csv', parse_dates=['dt_iso'])
#weather.head()# load in weather data

# change timestamp to date type
df['dt_iso_new'] = pd.to_datetime(df['dt_iso_new'])
df['dt_iso_new'] = df['dt_iso_new']#.apply(lambda x: x.replace(tzinfo=None))

# strip date and time and hour from timestamp
df['Time'] = df['dt_iso_new'].apply( lambda d : d.time())
df['Date'] = df['dt_iso_new'].apply( lambda d : d.date())
df['Hour'] = df['dt_iso_new'].apply( lambda d : d.hour)

# make date a date char type
df['Date'] = pd.to_datetime(df['Date'])
df.head()

#drop unecessary columns
df.drop(columns=['dt', 'dt_iso', 'timezone'], inplace=True)

# split weather data from 2010 through 2021
split_date ='2010-01-01'
weather_df = df.loc[df['Date'] >= split_date]

# concatenate date and hour together to form join key weather data
weather_df['Date'] = weather_df['Date'].astype(str)
weather_df['Hour'] = weather_df['Hour'].astype(str)
weather_df['date_join'] = weather_df[['Date', 'Hour']].apply(lambda x: ''.join(x), axis=1)

## Crimes over the last ~ 10 years

In [ ]:
# # group crimes by date and plot
import seaborn as sns

# set plot specs
sns.set(rc={'figure.figsize':(15, 7)})

# plot
df_date = df_incidents.groupby('Year_Month').count()
df_date.reset_index(inplace=True)
s = sns.lineplot(x='Year_Month', y='Offense', data=df_date)
s.set_xlabel('Year', fontsize=20)
s.set_ylabel('Count', fontsize=20)
s.axes.set_title('Crime Count by Month', fontsize=20)
s.tick_params(labelsize=11)
plt.show()

In [ ]:
# # group crimes by date and plot
import seaborn as sns

# set plot specs
sns.set(rc={'figure.figsize':(15, 7)})

# plot
df_date = df_incidents.groupby('Date').count()
df_date.reset_index(inplace=True)
s = sns.lineplot(x='Date', y='Offense', data=df_date)
s.set_xlabel('Date', fontsize=20)
s.set_ylabel('Count', fontsize=20)
s.axes.set_title('Crime Count by Day', fontsize=20)
s.tick_params(labelsize=11)
plt.show()

NameError: name 'df_incidents' is not defined

# Building the Model

In [ ]:
# concatenate date and hour together to form join key for crime data
df['Date'] = df['Date'].astype(str)
df['Hour'] = df['Hour'].astype(str)
df['date_join'] = df[['Date', 'Hour']].apply(lambda x: ''.join(x), axis=1)
df['Date'] = pd.to_datetime(df['Date'])

# join crime and weather data
df_new = df.merge(weather_df, on='date_join', how='left')
df_new.head(n=100)

In [ ]:
# join crime and weather data
df_new = df.merge(weather_df, on='date_join', how='left')

In [ ]:
# combine crimes into violent crime or not

df_new['violent_crime'] = np.where(df_new['Offense'].isin(['ARSON', 'ASLT1', 'ASLT2', 'ASLT3', 'ASLT4', 'CSCR', 'DASLT1'
                                                                     'DASLT2', 'DASLT3', 'DASTR', 'MURDR', 'ROBPAG', 
                                                                      'ROBPER', 'ROBBIZ', 'TFPER', 'JHOMIC']), 1, 0)

df_new['other_crime'] = np.where(~df_new['Offense'].isin(['ARSON', 'ASLT1', 'ASLT2', 'ASLT3', 'ASLT4', 'CSCR', 'DASLT1'
                                                                     'DASLT2', 'DASLT3', 'DASTR', 'MURDR', 'ROBPAG', 
                                                                      'ROBPER', 'ROBBIZ', 'TFPER', 'JHOMIC']), 1, 0)


In [ ]:
# subset df down to necessary columns
model_df = df_new.drop(['Time_x', 'Date_x', 'Year', 'DoW', 'Month_Name', 'ReportedDateTime', 'Neighborhood', 'Precinct', 'Offense', 'Description', 'date_join', 'dt', 'dt_iso',
            'dt_iso_new', 'timezone', 'city_name', 'lat', 'lon', 'feels_like', 'temp_min', 'temp_max',
           'pressure', 'sea_level', 'grnd_level', 'wind_deg', 'rain_3h', 
           #'rain_6h', 'rain_12h', 'rain_24h','rain_today', 
           'snow_3h', 
           # 'snow_6h', 'snow_12h', 'snow_24h', 'snow_today', 
           'clouds_all', 'weather_id',
           'weather_main', 'weather_description', 'weather_icon', 'Time_y', 'Date_y', 'Hour_y'], axis=1)

model_df['Hour_x'] = model_df['Hour_x'].astype(int)

# model_df['Month_Name'] = calendar.month_name[model_df['Month']]
# model_df['Month_Name'] = model_df['Month'].apply( lambda d : d.calendar.month_name() )

In [ ]:
# deal with nas

# count nas
# model_df.isna().sum()

# fill nas in cols
model_df = model_df.fillna(0)

model_df.tail()

In [ ]:
# look at correlations
# correlation heat map setup for seaborn
def corr_chart(df_corr):
    corr=df_corr.corr()
    #screen top half to get a triangle
    top = np.zeros_like(corr, dtype=bool)
    top[np.triu_indices_from(top)] = True
    fig=plt.figure()
    fig, ax = plt.subplots(figsize=(12,12))
    sns.heatmap(corr, mask=top, cmap='RdYlGn', 
        center = 0, square=True, 
        linewidths=.5, cbar_kws={'shrink':.5}, 
        annot = True, annot_kws={'size': 9}, fmt = '.3f')           
    plt.xticks(rotation=45) # rotate variable labels on columns (x axis)
    plt.yticks(rotation=0) # use horizontal variable labels on rows (y axis)
    plt.title('Correlation Heat Map')   
    plt.savefig('plot-corr-map.pdf', 
        bbox_inches = 'tight', dpi=None, facecolor='w', edgecolor='b', 
        orientation='portrait', format=None, 
        transparent=True, pad_inches=0.25)      

np.set_printoptions(precision=3)
corr_chart(df_corr = model_df)

##  Abstract

Brief description (300 words or less) of your analysis and results. 

## Introduction

Introduce your project. Describe all the processes you followed to solve the problem.

To have a perspective of the security states of the city of Minneapolis we defined few questions, which we answered them during our data analytic project.
Here is the list of these questions:
1. How has the number of various crimes changed over time in Minneapolis?
3. Are there any trends in the crimes being committed?
4. Which crimes are most frequently committed?
5. Which locations are these frequent crimes being committed to?
6. Are there certain high crime locations for certain crimes? 
7. How has the number of certain crimes (etc homicide) changed over the years in Minneapolis?
To answer these question we took the four main steps of the KDD data mining pipeline (see figure 1), which are respectively, data preprocessing, data pre-processing, analysis and post-processing.
In this documentation, we also use the same name for each section of the report. In section 2, we describe how we gathered our data and the tasks we did in regard to clean our data as data Pre-Processing phase. Section 3 we dive straight into the data analysis process, we first introduce the methods and technologies we used and then we provide details on how we dealt with crime data in Spark SQL. section 4 we bring the visualization of our results. Finally, in section 5 we bring the conclusion.



<blockquote class="citizen-incident-embed" data-video="true" data-details="true" data-background="false" data-remove-audio="true" data-responsive="true" data-video_stream_ids="mux:QPaIY7eoejpc01f02trIZZn01z3Nxf00Ob7LLhIAGVVU9oY"><p lang="en" dir="ltr">One Killed, Fourteen Injured in Shooting at Bar <a href="https://citizen.com">@CitizenApp</a></p>214 7th St W <a href="https://citizen.com/e/-Mlce2qWC8TiEZ6teu-Z">12:23:47 AM CDT</a><script async="" charset="utf-8" src="https://citizen.com/static/scripts/embed.js"></script><script type="text/javascript" async="" src="/static/scripts/embed.js?v=1633893811389"></script></blockquote>

## 1. The problem

What's the problem you are trying to solve?

[Back to top](#Table-of-contents)

## 2. Purpose of the study

Why are you solving the problem?

[Back to top](#Table-of-contents)

Problem Statements
Question 1: Which month has the most crime?
Question 2: Which weekday has the most crime?
Question 3: What time of day has the most crime?
Question 4: Where are crimes committed?
But first, let’s get the assumptions and methods out of the way.



The problem that this project will address is the following:
How to develop a machine learning model to conduct descriptive, predictive, and
prescriptive analysis of Minneapolis crime data?
The descriptive portion focuses on identifying spatial temporal relationships with
crime data. While the predictive focuses on potential future places and times of crimes.

In order to achieve it
system integrate Census data with the crime data and feed it to machine learning
algorithms.
In prescriptive analyzer it suggests process re-evaluate how to allocate
police resources most effeciently and effectively to reduce or deter crimes.

## 3. Project description

Describe your project.

[Back to top](#Table-of-contents)

### 3.1 Hypothesis

What are the hypothesis that drive your project?

[Back to top](#Table-of-contents)

### 3.2 Workflow

What's the process you are going to follow to solve the problem based on the Problem-Solving Framework:

- Define the problem 
- Map the system 
- Define suitable metrics 
- Select modeling tools 
- Implement model  
- Evaluate model 
- Deploy the final model 

[Back to top](#Table-of-contents)

### 3.3 Methods

What mathematical and computational methods are you using?

[Back to top](#Table-of-contents)

ML techniques used in crime prediction
In ref. [42], a comparative study was carried out between violent crime patterns from the Communities and Crime Unnormalized Dataset versus actual crime statistical data using the open source data mining software Waikato Environment for Knowledge Analysis (WEKA). Three algorithms, namely, linear regression, additive regression, and decision stump, were implemented using the same finite set of features on communities and actual crime datasets. Test samples were randomly selected. The linear regression algorithm could handle randomness to a certain extent in the test samples and thus proved to be the best among all three selected algorithms. The scope of the project was to prove the efficiency and accuracy of ML algorithms in predicting violent crime patterns and other applications, such as determining criminal hotspots, creating criminal profiles, and learning criminal trends.

### 3.4 Analysis 

Spend some time analysing your models, data and processes.

[Back to top](#Table-of-contents)

### 3.5 Results

What are your results?

[Back to top](#Table-of-contents)

## Conclusion

What are your conclusions?

[Back to top](#Table-of-contents)

## References

Add references and bibliography you used in the project.

Use this as an example for books:

- Bishop, Christopher M. 2006. *Pattern Recognition and Machine Learning*. Springer.

Use this as an example for articles:

- Bubeck, Sébastien. 2015. Convex Optimization: Algorithms and Complexity. *Foundations and Trends in Machine Learning*, **8**(3-4), 231–357.

[Back to top](#Table-of-contents)

### Field Descriptions - PIMS (06/01/2018 to present)

- **BeginDate** Date incident began. Time in the field is UTC not local, so a separate column is created for accurate time information.
- **Time** Begin date time field.
- **CCN** A concatenation of the 4 digit year in which the incident was created, followed by a dash and then a 6 digit number of sequence for the agency. The MP at the beginning signifies a report taken by Minneapolis Police. This is used because Minneapolis shares the record management system with the University of 
- **Minnesota Police ControlNbr** A unique identifier for case.
- **Offense** Code of criminal act reported.
- **Description** Description of the criminal code of incident.
- **EnteredDate** The timestamp of when the incident was created in the system.
- **GBSID** The anonymized street centerline ID.
- **LastChanged** Date the record was last altered in system.
- **LastUpdateDate** Date the record was last moved to open data.
- **Lat** The anonymized latitude of the incident.
- **Long** The anonymized longitude of the incident.
- **Neighborhood** The neighborhood of the incident. Note that occasionally due to the anonymization process, if a point is on the boundary of a neighborhood, it may fall into either neighborhood.
- **OBJECTID** A unique identifier for open data portal.
- **Precinct** The police precinct of the incident. Note that occasionally due to the anonymization process, if a point is on the boundary of a precinct, it may fall into either precinct.
- **PublicAddress** Address of incident anonymized to the block.
- **ReportedDate** Date incident is reported to police.

- **Categoy** = UCRCode: Code that signifies the type of crime that was committed. (1 = MURDER3 = RAPE 4 = ROBBERY 5 = ASSAULT 6 = BURGLARY 7 = LARCENY 8 = AUTO THEFT 10 = ARSON)

### Field Descriptions 

- CAPRS (01/01/2010 to present) *The system is 28 years old but 2010 is the oldest data pulbished on https://opendata.minneapolismn.gov/
-
- **PublicAddress** (type: esriFieldTypeString, alias: PublicAddress, SQL Type: sqlTypeOther, length: 100, nullable: true, editable: true)
- **ControlNbr** (type: esriFieldTypeInteger, alias: ControlNbr, SQL Type: sqlTypeOther, nullable: false, editable: true)
- **CCN** (type: esriFieldTypeString, alias: CCN, SQL Type: sqlTypeOther, length: 28, nullable: true, editable: true)
- **Precinct** (type: esriFieldTypeString, alias: Precinct, SQL Type: sqlTypeOther, length: 2, nullable: true, editable: true)
- **ReportedDate** (type: esriFieldTypeDate, alias: ReportedDate, SQL Type: sqlTypeOther, length: 8, nullable: true, editable: true)
- **BeginDate** (type: esriFieldTypeDate, alias: BeginDate, SQL Type: sqlTypeOther, length: 8, nullable: true, editable: true)
- **Time** (type: esriFieldTypeString, alias: Time, SQL Type: sqlTypeOther, length: 20, nullable: true, editable: true)
- **Offense** (type: esriFieldTypeString, alias: Offense, SQL Type: sqlTypeOther, length: 20, nullable: false, editable: true)
- **Description** (type: esriFieldTypeString, alias: Description, SQL Type: sqlTypeOther, length: 65, nullable: true, editable: true)
- **UCRCode** (type: esriFieldTypeString, alias: UCRCode, SQL Type: sqlTypeOther, length: 3, nullable: true, editable: true)
- **EnteredDate** (type: esriFieldTypeDate, alias: EnteredDate, SQL Type: sqlTypeOther, length: 8, nullable: true, editable: true)
- **GBSID** (type: esriFieldTypeDouble, alias: GBSID, SQL Type: sqlTypeOther, nullable: true, editable: true)
- **Lat** (type: esriFieldTypeDouble, alias: Lat, SQL Type: sqlTypeOther, nullable: true, editable: true)
- **Long** (type: esriFieldTypeDouble, alias: Long, SQL Type: sqlTypeOther, nullable: true, editable: true)
- **X** (type: esriFieldTypeDouble, alias: X, SQL Type: sqlTypeOther, nullable: true, editable: true)
- **Y** (type: esriFieldTypeDouble, alias: Y, SQL Type: sqlTypeOther, nullable: true, editable: true)
- **Neighborhood** (type: esriFieldTypeString, alias: Neighborhood, SQL Type: sqlTypeOther, length: 32, nullable: true, editable: true)
- **LastChanged** (type: esriFieldTypeDate, alias: LastChanged, SQL Type: sqlTypeOther, length: 8, nullable: true, editable: true)
- **LastUpdateDate** (type: esriFieldTypeDate, alias: LastUpdateDate, SQL Type: sqlTypeOther, length: 8, nullable: true, editable: true)
- **ESRI_OID** (type: esriFieldTypeOID, alias: ESRI_OID, SQL Type: sqlTypeOther, length: 0, nullable: false, editable: false)
